# Exciton Decomposition Analysis

There are qualititative and quantative analysis, for qualititive you can use summarize eigenvectors, just know the largest band to band transition and largest k point, then you will learn the band composition into parts.

For ploteigenvectors.py is also a qualititive analysis, summarize eigenvectors is also semi-qualiative, because of limited information: 

It only tells you about wtot, wmax and ikmax for different band to band transitions for each Exciton S - You cannot resolve K-point from this information, thus cannot safely project to atomic orbitals.

`wtot = sum_k |A_vck|^2. wmax = max_k |A_vck|^2. |A_vc (ikmax)|^2 = wmax.`


**To do a proper quantitative approach:**

Read eigenvectors.h5 to extract K resolved decomposition into different band to band transition, in combination with pdos information, I can calculate the percentage for perovskite/non-perovskite of electrons and holes for the every excitons, similar information are also calculated when I did the ECF-noshift plot, where I can see the localization of electrons and holes for each exciton in real space.

file:///Users/yinanchen/PhD/2022_Intergrowth_Hema/APD1_Na/absorption/k104/evec_component_k=0.png
file:///Users/yinanchen/PhD/2022_Intergrowth_Hema/APD1_Na/absorption/k104/evec_component_k=-1.png
file:///Users/yinanchen/PhD/2022_Intergrowth_Hema/APD2_Pb/corrected_H/absorption/evec_component_k=-1.png

In [213]:
import pandas as pd
import numpy as np

In [214]:
# Things to modify by hand

kgrid_file = 'kgrid.log_cocn.bak'
kgrid_file = 'kgrid.log_CYS.bak'
kgrid_file = 'kgrid.log_APD2.bak'
kgrid_file = 'kgrid.log_APD1.bak'

bandstructure_file = 'bandstructure.dat_cocn.bak'
bandstructure_file = 'bandstructure.dat_CYS.bak'
bandstructure_file = 'bandstructure.dat_APD2.bak'
bandstructure_file = 'bandstructure.dat_APD1.bak'

fermi = 280 # ! for HOCN
fermi = 304   # ! for COCN
fermi = 544   # ! for CYS
fermi = 408 # ! VBM - start + 1 , hard-coded for APD2_Li now, read from projbands file
fermi = 432   # ! for APD1_Na


eigenvector_file = 'eigenvectors.h5_hocn.bak'
eigenvector_file = 'eigenvectors.h5_cocn.bak'
eigenvector_file = 'eigenvectors.h5.CYS.bak'
eigenvector_file = 'eigenvectors.h5.APD2.bak'
eigenvector_file = 'eigenvectors.h5.APD1.bak'
eigenvector_file = 'eigenvectors_APD1_relax.h5'

scf_file = 'relax.out_cocn.bak'
scf_file = 'scf.out_CYS.bak'
scf_file = 'scf.out_APD2_Li.bak'
scf_file = 'scf.out_APD1.bak'

projbands_file = 'cl.projbands_hocn.bak'
projbands_file = 'cl.projbands_cocn.bak'
projbands_file = 'cl.projbands.CYS.bak'
projbands_file = 'cl.projbands_APD2_Li.bak'
projbands_file = 'cl.projbands.apd1.bak'
projbands_file = 'cl.projbands_APD1_relax.bak'

e_min = 0
e_high = 5

## Step 0: Find the K list correspondance from bandstructure.dat (unfolded) with the K list from kgrid.log (folded)

### 0.1 Process K list from Kgrid.log (folded)

In [215]:
## Process the kgrid.log, find the correpondance with uniform grid and irrduciible grid used to calculate WFN_fi

def process_kgrid_WFN(file_content):
    lines = file_content.split('\n')

    for i, line in enumerate(lines):
        if "k-points in the original uniform grid" in line:
            # Next line contains the number of rows to read
            num_rows = int(lines[i + 1].strip())
            # Data starts from the next line
            start_line = i + 2
            break

    data = []
    for line in lines[start_line:start_line + num_rows]:
        # Split the line into columns and convert to appropriate types
        columns = line.split()
        row = [int(columns[0])] + [float(c) for c in columns[1:5]] + [int(columns[5]), columns[6]]
        data.append(row)
    unfolded_idx = {}
    
    cnt = 0
    processed_data = []

    for row in data:
        row_number = row[0]
        reference_row = row[5]

        # If the sixth column is 0, increment the count
        if reference_row == 0:
            unfolded_idx[row_number] = cnt
            cnt = cnt + 1
        else:
            # If the sixth column is not 0, set the count to the count of the referenced row
            unfolded_idx[row_number] = unfolded_idx[reference_row]
        processed_data.append(row + [unfolded_idx[row_number]])

    return processed_data

processed_data = process_kgrid_WFN(open(kgrid_file).read())
# the last column and kx, ky, kz
processed_data = np.array(processed_data)[:, [1, 2, 3, 7]].astype(float)
nk = processed_data.shape[0]
processed_data

array([[0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00],
       [0.00e+00, 0.00e+00, 2.50e-01, 1.00e+00],
       [0.00e+00, 0.00e+00, 5.00e-01, 2.00e+00],
       ...,
       [9.00e-01, 9.00e-01, 2.50e-01, 3.97e+02],
       [9.00e-01, 9.00e-01, 5.00e-01, 3.98e+02],
       [9.00e-01, 9.00e-01, 7.50e-01, 3.99e+02]])

### 0.2 Process bandstructure.dat (unfolded)

In [216]:
def read_unfolded(file_path):
    matrix = []
    last_spin, last_band = None, None

    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('#'):
                continue

            parts = line.split()
            if len(parts) < 5:
                continue  
            spin, band, kx, ky, kz = parts[:5]

            # check if we have a new spin or band
            if last_spin is not None and (spin != last_spin or band != last_band):
                break
            # update last spin and band
            last_spin, last_band = spin, band

            matrix.append([float(kx), float(ky), float(kz)])

    return np.array(matrix)

data_matrix = read_unfolded(bandstructure_file)
print(data_matrix.shape)


(400, 3)


In [217]:

#reciprocal_lattice_vectors and alat are read from relax.out or scf.out

def extract_specific_data(file_path):
    with open(file_path, 'r') as file:
        alat = None
        b_vectors = []

        for line in file:
            if 'lattice parameter (alat)' in line:
                parts = line.split('=')
                if len(parts) >= 2:
                    alat = float(parts[1].split()[0])

            if 'reciprocal axes' in line:
                for i in range(3):
                    b_vectors.append(file.readline().split()[3:6])
            # ! Heavily depends on the format of the file
            if alat is not None and len(b_vectors) == 3:
                break

    return alat, np.array(b_vectors, dtype=float)

alat, b_vectors = extract_specific_data(scf_file)
print(alat)
print(b_vectors)
two_pi_over_alat = 2 * np.pi / alat
reciprocal_lattice_vectors = np.array(b_vectors) * two_pi_over_alat
reciprocal_lattice_vectors

15.4338
[[ 1.       -0.        0.040671]
 [ 0.        1.05533  -0.      ]
 [ 0.        0.        0.349423]]


array([[ 0.40710553, -0.        ,  0.01655739],
       [ 0.        ,  0.42963068, -0.        ],
       [ 0.        ,  0.        ,  0.14225204]])

In [218]:
# Convert cartesian coordinates to fractional coordinates
def cartesian_to_fractional(cartesian_coordinates, reciprocal_lattice_vectors):
    return np.dot(cartesian_coordinates, np.linalg.inv(reciprocal_lattice_vectors))

def fractional_to_cartesian(fractional_coordinates, reciprocal_lattice_vectors):
    return np.dot(fractional_coordinates, reciprocal_lattice_vectors)

fractional_coordinates = cartesian_to_fractional(data_matrix, reciprocal_lattice_vectors)
# Around to 3 decimals
fractional_coordinates = np.around(fractional_coordinates, decimals=3)
fractional_coordinates

array([[ 0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.05,  0.  ],
       [ 0.  ,  0.1 ,  0.  ],
       ...,
       [ 0.5 , -0.15,  0.  ],
       [ 0.5 , -0.1 ,  0.  ],
       [ 0.5 , -0.05,  0.  ]])

In [219]:
# Convert fractional coordinates to the BZ within (0,0,0) to (1,1,1) by translating the coordinates
def translate_to_BZ(fractional_coordinates):
    return fractional_coordinates - np.floor(fractional_coordinates)

translated_coordinates = translate_to_BZ(fractional_coordinates)
translated_coordinates[:20]

array([[0.  , 0.  , 0.  ],
       [0.  , 0.05, 0.  ],
       [0.  , 0.1 , 0.  ],
       [0.  , 0.15, 0.  ],
       [0.  , 0.2 , 0.  ],
       [0.  , 0.25, 0.  ],
       [0.  , 0.3 , 0.  ],
       [0.  , 0.35, 0.  ],
       [0.  , 0.4 , 0.  ],
       [0.  , 0.45, 0.  ],
       [0.  , 0.5 , 0.  ],
       [0.  , 0.55, 0.  ],
       [0.  , 0.6 , 0.  ],
       [0.  , 0.65, 0.  ],
       [0.  , 0.7 , 0.  ],
       [0.  , 0.75, 0.  ],
       [0.  , 0.8 , 0.  ],
       [0.  , 0.85, 0.  ],
       [0.  , 0.9 , 0.  ],
       [0.  , 0.95, 0.  ]])

In [220]:
# Sort the translated coordinates first by kx, then by ky, then by kz, and keeping the original indices
def sort_by_k(translated_coordinates):
    indices = np.lexsort((translated_coordinates[:,2], translated_coordinates[:,1], translated_coordinates[:,0]))
    return translated_coordinates[indices], indices

sorted_coordinates, indices = sort_by_k(translated_coordinates)
print(sorted_coordinates[:20])
print(indices[:20])

[[0.   0.   0.  ]
 [0.   0.05 0.  ]
 [0.   0.1  0.  ]
 [0.   0.15 0.  ]
 [0.   0.2  0.  ]
 [0.   0.25 0.  ]
 [0.   0.3  0.  ]
 [0.   0.35 0.  ]
 [0.   0.4  0.  ]
 [0.   0.45 0.  ]
 [0.   0.5  0.  ]
 [0.   0.55 0.  ]
 [0.   0.6  0.  ]
 [0.   0.65 0.  ]
 [0.   0.7  0.  ]
 [0.   0.75 0.  ]
 [0.   0.8  0.  ]
 [0.   0.85 0.  ]
 [0.   0.9  0.  ]
 [0.   0.95 0.  ]]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


In [221]:
# append the indices to the processed_data
processed_kgrid_wfn_eigenvector = np.hstack((processed_data, indices.reshape(-1,1)))
processed_kgrid_wfn_eigenvector[:20]

array([[ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.25,  1.  ,  1.  ],
       [ 0.  ,  0.  ,  0.5 ,  2.  ,  2.  ],
       [ 0.  ,  0.  ,  0.75,  3.  ,  3.  ],
       [ 0.  ,  0.1 ,  0.  ,  4.  ,  4.  ],
       [ 0.  ,  0.1 ,  0.25,  5.  ,  5.  ],
       [ 0.  ,  0.1 ,  0.5 ,  6.  ,  6.  ],
       [ 0.  ,  0.1 ,  0.75,  7.  ,  7.  ],
       [ 0.  ,  0.2 ,  0.  ,  8.  ,  8.  ],
       [ 0.  ,  0.2 ,  0.25,  9.  ,  9.  ],
       [ 0.  ,  0.2 ,  0.5 , 10.  , 10.  ],
       [ 0.  ,  0.2 ,  0.75, 11.  , 11.  ],
       [ 0.  ,  0.3 ,  0.  , 12.  , 12.  ],
       [ 0.  ,  0.3 ,  0.25, 13.  , 13.  ],
       [ 0.  ,  0.3 ,  0.5 , 14.  , 14.  ],
       [ 0.  ,  0.3 ,  0.75, 15.  , 15.  ],
       [ 0.  ,  0.4 ,  0.  , 16.  , 16.  ],
       [ 0.  ,  0.4 ,  0.25, 17.  , 17.  ],
       [ 0.  ,  0.4 ,  0.5 , 18.  , 18.  ],
       [ 0.  ,  0.4 ,  0.75, 19.  , 19.  ]])

In [222]:
# Check both indices are correct, the first one should be foled index, the second one is the unfolded index
print(max(processed_kgrid_wfn_eigenvector[:,4]))
print(max(processed_kgrid_wfn_eigenvector[:,3]))
print(len(processed_kgrid_wfn_eigenvector))

399.0
399.0
400


## Step 1: Decomposition of excitons into the contribution of band-to-band transitions at different k points

Specifically, I read and store sum_v/c |Acvk|^2 for each (S,k) pair for each band in c, v respectively. Then I can also plot the distribution of the contribution of each band-to-band transition to the exciton wavefunction as I did using the script plot_eigenvectors_kloop.py.

file:///Users/yinanchen/PhD/2022_Intergrowth_Hema/APD1_Na/absorption/k104/evec_component_k=0.png
file:///Users/yinanchen/PhD/2022_Intergrowth_Hema/APD1_Na/absorption/k104/evec_component_k=-1.png

In [223]:
with open(projbands_file, 'r') as f:
    lines = f.readlines()

comments = []
for line in lines:
    if line.startswith('#'):
        comments.append(line)
    else:
        break

data = []
for line in lines:
    if not line.startswith('#'):
        data.append(line)


In [224]:
# convert the list of strings to a list of lists, 2D array
data = [line.split() for line in data]

In [225]:
# conver the 2d array to a dataframe, and convert the strings to floats
df = pd.DataFrame(data).astype(float)

In [226]:
df

,0,1,2,3,4,5,6,7,8,9,...,674,675,676,677,678,679,680,681,682,683
0,1.0,0.00000,-50.50786,1.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.00000,-50.50786,1.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.00000,-50.50777,0.992,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.00000,-50.50777,0.992,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.00000,-50.50774,0.996,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195995,400.0,60.33289,5.82564,0.556,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
195996,400.0,60.33289,5.83856,0.584,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.001,0.001,0.0,0.0,0.0,0.0,0.0,0.0
195997,400.0,60.33289,5.87085,0.628,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
195998,400.0,60.33289,5.88086,0.544,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.002,0.002,0.0,0.0,0.0,0.0,0.0,0.0


In [227]:
import h5py
with h5py.File(eigenvector_file, 'r') as f:
    evals_shape = f['exciton_data/eigenvalues'].shape
    print("Eigenvalues shape:", evals_shape)
    
    evc_shape = f['exciton_data/eigenvectors'].shape
    print("Eigenvectors shape:", evc_shape)


Eigenvalues shape: (10,)
Eigenvectors shape: (1, 10, 400, 20, 20, 1, 2)


### Read only the partial information from eigenvectors.h5

In [173]:
import numpy as np
import h5py as h5
import matplotlib.pyplot as plt

contrib_c_dict = {}
contrib_v_dict = {}

def plot_for_k(eigenvector_file, k_select, iN_S_select, e_min=None, e_high=None):
    with h5.File(eigenvector_file, 'r') as f:
        # 只读取特定iN_S的eigenvalues
        evals = f['exciton_data/eigenvalues'][iN_S_select]
        # 只读取特定iN_S的eigenvectors
        evc = f['exciton_data/eigenvectors'][0,iN_S_select,:,:,:,0,:]
        nk, nc, nv, _ = evc.shape

        if k_select != -1:
            evc = evc[[k_select],:,:,:]

        if e_min<evals<e_high:
            temp_contrib_cv = np.sum(abs(evc[:,:,:,0]+evc[:,:,:,1]*1j)**2, axis=0) # Summarize_eigenvectors and Ploteigenvectors - For a certain k and c, calculating the sum of Acvk across all vs, and vice versa, this is the correct way to do it, because for a certain k, valence band pdos idoes not change when we fix a v, and vice versa, But you ignore the correlation between c and v, thus you miss the ECF, but you can only plot ECF_noshift
            temp_contrib_v = np.sum(temp_contrib_cv, axis=0)
            temp_contrib_c = np.sum(temp_contrib_cv, axis=1)

            contrib_c_dict[(iN_S_select,k_select)] = temp_contrib_c
            contrib_v_dict[(iN_S_select,k_select)] = temp_contrib_v
        
        return 1, nk, nc, nv  # 由于只读取一个iN_S，nS始终为1

In [174]:
iN_S_select =1
for k in range(0,nk):
    nS, nk, nc, nv = plot_for_k(eigenvector_file,k,iN_S_select,0,5)
    #print(contrib_c_dict)
    #print(contrib_v_dict)
print(nk, nc, nv, nS)

400 20 20 1


### Read the full information from eigenvectors.h5

In [228]:
import numpy as np
import h5py as h5
import matplotlib.pyplot as plt
contrib_c_dict = {}
contrib_v_dict = {}

def plot_for_k(k_select):
    f = h5.File(eigenvector_file,'r')
    evals = f['exciton_data/eigenvalues'][()]
    evc = f['exciton_data/eigenvectors'][()]
    evc = evc[0,:,:,:,:,0,:]
    (nS, nk, nc, nv, _) = evc.shape

    if k_select == -1:
        pass
    else:
        evc = evc[:,[k_select],:,:,:]

    for iN_S in range(nS):
        if e_min<evals[iN_S]<e_high:
            temp_contrib_cv = np.sum(abs(evc[iN_S,:,:,:,0]+evc[iN_S,:,:,:,1]*1j)**2, axis=0) # Summarize_eigenvectors and Ploteigenvectors - For a certain k and c, calculating the sum of Acvk across all vs, and vice versa, this is the correct way to do it, because for a certain k, valence band pdos idoes not change when we fix a v, and vice versa, But you ignore the correlation between c and v, thus you miss the ECF, but you can only plot ECF_noshift
            temp_contrib_v = np.sum(temp_contrib_cv, axis=0)
            temp_contrib_c = np.sum(temp_contrib_cv, axis=1)

            contrib_c_dict[(iN_S,k_select)] = temp_contrib_c
            contrib_v_dict[(iN_S,k_select)] = temp_contrib_v
    return nS, nk, nc, nv

In [229]:
for k in range(0,nk):
    nS, nk, nc, nv = plot_for_k(k)
    #print(contrib_c_dict)
    #print(contrib_v_dict)
print(nk)

400


In [230]:
contrib_c_dict

{(0,
  0): array([4.59134929e-02, 4.49464307e-02, 2.87861725e-08, 2.67766220e-08,
        2.43658653e-08, 2.15494123e-08, 1.29246520e-09, 2.06974253e-09,
        7.55297738e-09, 6.32734140e-09, 6.91072263e-08, 7.07844868e-08,
        3.73018817e-07, 4.72864306e-07, 2.24751839e-08, 2.29740875e-08,
        5.96256209e-10, 5.63277104e-10, 9.26826351e-09, 9.49038156e-09]),
 (1,
  0): array([6.27507930e-02, 3.96913815e-02, 2.99172219e-08, 2.35916689e-08,
        6.74767013e-06, 4.48624126e-06, 1.51288578e-09, 2.19601117e-09,
        1.73368029e-08, 2.72319810e-08, 3.76711787e-08, 5.49824683e-08,
        1.27062910e-06, 7.30663993e-07, 1.67636730e-08, 1.28781658e-08,
        3.87653902e-10, 4.32513300e-10, 7.62800130e-09, 4.47358062e-09]),
 (2,
  0): array([4.02619025e-02, 6.42246183e-02, 2.12461244e-08, 2.45232861e-08,
        4.88275563e-06, 7.17302222e-06, 1.17742566e-09, 5.86542848e-10,
        2.35639990e-08, 1.85901424e-08, 4.84447122e-08, 3.12442364e-08,
        6.84468261e-07, 1.1622

In [231]:
nS, nk, nc, nv

(10, 400, 20, 20)

In [232]:
# find the corresponding pdos for each k from df
# Count the number of k points in df, should be consistent with nk
n_k = len(df[0].unique())
n_k

400

## Step 2: Calculate the kpdos on a WFN_fi calculation

Now that we have the decomposition of excitons into the contribution of band-to-band transitions for each (S,k) pair.

For every k, find the corresponding pdos from K-resolved Projected DOS on WFN_fi.

In [233]:
# Before that, we need to isolating only the middle nv+nc rows for each k, 
# corresponding to the nv valence and nc conduction bands

# First group df by the first column, then operate on each group
# For each group, take out the middle nv+nc rows around fermi, i.e. fermi - nv : fermi + nc
# ! Locate the position of fermi, i.e. the position where df[2] changes from negative to positive. 

conduction_df = df.groupby(0).apply(lambda x: x.iloc[fermi:fermi+nc,:])
valence_df = df.groupby(0).apply(lambda x: x.iloc[fermi-nv:fermi,:])
reversed_valence_df = valence_df.iloc[::-1,:]
reversed_valence_df.shape

(8000, 684)

In [234]:
# This step is to check the vs are negative and cs are positive
# To ensure that the hard-coded fermi is correct

valence_df.loc[1]
print(valence_df.loc[1][2].values)
print(reversed_valence_df.loc[1][2].values)
print(conduction_df.loc[1][2].values)

[-1.37077e+00 -1.37077e+00 -1.37046e+00 -1.37046e+00 -1.20203e+00
 -1.20203e+00 -1.11908e+00 -1.11908e+00 -1.08883e+00 -1.08883e+00
 -8.94960e-01 -8.94960e-01 -8.85420e-01 -8.85420e-01 -6.11120e-01
 -6.11120e-01 -3.56130e-01 -3.56130e-01  5.00000e-05  5.00000e-05]
[ 5.00000e-05  5.00000e-05 -3.56130e-01 -3.56130e-01 -6.11120e-01
 -6.11120e-01 -8.85420e-01 -8.85420e-01 -8.94960e-01 -8.94960e-01
 -1.08883e+00 -1.08883e+00 -1.11908e+00 -1.11908e+00 -1.20203e+00
 -1.20203e+00 -1.37046e+00 -1.37046e+00 -1.37077e+00 -1.37077e+00]
[1.84173 1.84173 2.62054 2.62054 3.27624 3.27624 3.66066 3.66066 3.84474
 3.84474 3.8677  3.8677  3.91501 3.91501 3.98572 3.98572 4.25262 4.25262
 4.28125 4.28125]


In [235]:
# To correspond to the contribution of valence bands (counting from the VBM) and conduction bands (counting from the CBM)
# We need to reverse the order of the valence bands

reversed_valence_df.loc[41]
conduction_df.loc[41]


,0,1,2,3,4,5,6,7,8,9,...,674,675,676,677,678,679,680,681,682,683
20032,41.0,6.1316,1.88526,0.944,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.0,0.000,0.000
20033,41.0,6.1316,2.00317,0.948,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.0,0.000,0.000
20034,41.0,6.1316,2.49778,0.940,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.0,0.000,0.000
20035,41.0,6.1316,2.80513,0.944,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.0,0.000,0.000
20036,41.0,6.1316,3.30523,0.924,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.0,0.000,0.000
20037,41.0,6.1316,3.42487,0.936,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.0,0.000,0.000
20038,41.0,6.1316,3.57936,0.884,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.0,0.000,0.000
20039,41.0,6.1316,3.71678,0.848,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.0,0.000,0.000
20040,41.0,6.1316,3.75847,0.888,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.0,0.000,0.000
20041,41.0,6.1316,3.83919,0.819,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.0,0.000,0.000


In [236]:
# Normalize the reversed_valence_df and conduction_df by the sum of the pdos of each k (the 3rd column)

reversed_valence_df = reversed_valence_df.apply(lambda x: x/x[3], axis=1)
conduction_df = conduction_df.apply(lambda x: x/x[3], axis=1)
reversed_valence_df.loc[41]
conduction_df.loc[41]

,0,1,2,3,4,5,6,7,8,9,...,674,675,676,677,678,679,680,681,682,683
20032,43.432203,6.495339,1.997097,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
20033,43.248945,6.467932,2.113049,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
20034,43.617021,6.522979,2.657213,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
20035,43.432203,6.495339,2.971536,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
20036,44.372294,6.635931,3.577089,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
20037,43.803419,6.550855,3.659049,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
20038,46.380090,6.936199,4.049050,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
20039,48.349057,7.230660,4.382995,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
20040,46.171171,6.904955,4.232511,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
20041,50.061050,7.486691,4.687656,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000


## Step 3: Combine the knowledge of K resolved pdos and exciton decomposition into bands

For each k point, find out the decomposition of bands into atomic orbitals

In [237]:
# Now we use contrib_v_dict[(s,k)] as the weight to multiply with the pdos of the valence band at k = k_select
# Then we sum over all k points, and we get the pdos of the valence band

# ! Note, Question: should we normalize the weight? i.e. divided by the sum of the weights over all vs or cs for each k point?
# ! Answer: No, because each k point in BZ should have a different total weight as seen by the plot_eigenvectors.py plots,
# #! so we should not normalize the weight for each k point.

# ! Also, I am not normalizing pdos (not divided by TOTAL (ranging from 0.7 - 0.9) in column 4), 
# ! using their original pdos at (k, n), where n is the band index.
# ! We should normalize the pdos by the TOTAL
iN_S_select
result_df_valence = pd.DataFrame()
result_df_conduction = pd.DataFrame()
for k_select in range(len(processed_kgrid_wfn_eigenvector)):
    k_select_kpdos =  int(processed_kgrid_wfn_eigenvector[k_select][3])
    k_select_eigenvector = int(processed_kgrid_wfn_eigenvector[k_select][4])
    weighted_sum_valence = reversed_valence_df.loc[k_select_kpdos+1].mul(contrib_v_dict[(iN_S_select
,k_select_eigenvector)], axis=0).sum()
    # \ / contrib_v_dict[(iN_s_select,k_select)].sum()
    result_df_weighted_sum_valence = pd.DataFrame([weighted_sum_valence])

    weighted_sum_conduction = conduction_df.loc[k_select_kpdos+1].mul(contrib_c_dict[(iN_S_select
,k_select_eigenvector)], axis=0).sum()
    # \ / contrib_c_dict[(iN_s_select,k_select)].sum()
    result_df_weighted_sum_conduction = pd.DataFrame([weighted_sum_conduction])

    # append all k points into a single dataframe
    #result_df = result_df.append(result_df_weighted_sum)
    result_df_valence = pd.concat([result_df_valence, result_df_weighted_sum_valence], axis=0)
    result_df_conduction = pd.concat([result_df_conduction, result_df_weighted_sum_conduction], axis=0)

# sum over all k points, row-wise
print(result_df_valence.sum(axis=0))
print(result_df_conduction.sum(axis=0))
result_df_conduction

0      113.247722
1       16.827142
2       -0.247772
3        1.000000
4        0.000000
          ...    
679      0.000000
680      0.000000
681      0.000000
682      0.000000
683      0.000000
Length: 684, dtype: float64
0      1.109025e+02
1      1.647134e+01
2      2.279004e+00
3      1.000000e+00
4      0.000000e+00
           ...     
679    5.571352e-08
680    5.571352e-08
681    0.000000e+00
682    1.653340e-08
683    1.653340e-08
Length: 684, dtype: float64


,0,1,2,3,4,5,6,7,8,9,...,674,675,676,677,678,679,680,681,682,683
0,0.107622,0.000000,0.198233,0.102456,0.0,0.0,0.0,0.0,0.0,0.0,...,1.265690e-12,1.265690e-12,3.667772e-11,3.667772e-11,0.0,2.531380e-12,2.531380e-12,0.0,1.265690e-12,1.265690e-12
0,0.213124,0.009285,0.196284,0.101447,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.000000e+00,9.159730e-10,9.159730e-10,0.0,6.458590e-11,6.458590e-11,0.0,0.000000e+00,0.000000e+00
0,0.320078,0.018592,0.196526,0.101571,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.000000e+00,1.441268e-09,1.441268e-09,0.0,1.076299e-10,1.076299e-10,0.0,0.000000e+00,0.000000e+00
0,0.417586,0.027288,0.192294,0.099385,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.000000e+00,2.893747e-10,2.893747e-10,0.0,3.158046e-11,3.158046e-11,0.0,0.000000e+00,0.000000e+00
0,0.106206,0.011534,0.041357,0.020180,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.000000e+00,4.283170e-09,4.283170e-09,0.0,2.690171e-10,2.690171e-10,0.0,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,3.851897,0.581577,0.021714,0.009249,0.0,0.0,0.0,0.0,0.0,0.0,...,5.162542e-12,5.162542e-12,1.516985e-09,1.516985e-09,0.0,2.118577e-10,2.118577e-10,0.0,5.162542e-12,5.162542e-12
0,2.497695,0.377935,0.012759,0.005960,0.0,0.0,0.0,0.0,0.0,0.0,...,4.374531e-12,4.374531e-12,3.665191e-10,3.665191e-10,0.0,3.276190e-11,3.276190e-11,0.0,4.374531e-12,4.374531e-12
0,3.897017,0.589043,0.019848,0.009277,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000e+00,0.000000e+00,5.721920e-10,5.721920e-10,0.0,3.311919e-11,3.311919e-11,0.0,0.000000e+00,0.000000e+00
0,0.467869,0.070644,0.002406,0.001109,0.0,0.0,0.0,0.0,0.0,0.0,...,1.914410e-11,1.914410e-11,1.873710e-09,1.873710e-09,0.0,8.606934e-11,8.606934e-11,0.0,1.914410e-11,1.914410e-11


In [238]:
ns_accumulated_df_valence = pd.DataFrame()
ns_accumulated_df_conduction = pd.DataFrame()
#for s_select in range(nS):
for s_select in range(iN_S_select,iN_S_select+1):
    accumulated_df_valence = pd.DataFrame()
    accumulated_df_conduction = pd.DataFrame()
    for k_select in range(len(processed_kgrid_wfn_eigenvector)):
        k_select_kpdos = int(processed_kgrid_wfn_eigenvector[k_select][3])
        k_select_eigenvector = int(processed_kgrid_wfn_eigenvector[k_select][4])
        current_weighted_sum_valence = reversed_valence_df.loc[k_select_kpdos+1].mul(contrib_v_dict[(s_select, k_select_eigenvector)], axis=0).sum()
        current_weighted_sum_conduction = conduction_df.loc[k_select_kpdos+1].mul(contrib_c_dict[(s_select, k_select_eigenvector)], axis=0).sum()
        if accumulated_df_valence.empty:
            accumulated_df_valence = pd.DataFrame([current_weighted_sum_valence])
        else:
            accumulated_df_valence += current_weighted_sum_valence
        if accumulated_df_conduction.empty:
            accumulated_df_conduction = pd.DataFrame([current_weighted_sum_conduction])
        else:
            accumulated_df_conduction += current_weighted_sum_conduction
    ns_accumulated_df_valence = pd.concat([ns_accumulated_df_valence, accumulated_df_valence], axis=0)
    ns_accumulated_df_conduction = pd.concat([ns_accumulated_df_conduction, accumulated_df_conduction], axis=0)

print(accumulated_df_conduction)
#result_df
ns_accumulated_df_conduction


          0          1         2    3    4    5    6    7    8    9    ...  \
0  110.902454  16.471339  2.279004  1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   

            674           675           676           677  678           679  \
0  1.633357e-08  1.633357e-08  6.748456e-07  6.747069e-07  0.0  5.571352e-08   

            680  681           682           683  
0  5.571352e-08  0.0  1.653340e-08  1.653340e-08  

[1 rows x 684 columns]


,0,1,2,3,4,5,6,7,8,9,...,674,675,676,677,678,679,680,681,682,683
0,110.902454,16.471339,2.279004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.633357e-08,1.633357e-08,6.748456e-07,6.747069e-07,0.0,5.571352e-08,5.571352e-08,0.0,1.653340e-08,1.653340e-08


In [239]:
ns_accumulated_df_valence

,0,1,2,3,4,5,6,7,8,9,...,674,675,676,677,678,679,680,681,682,683
0,113.247722,16.827142,-0.247772,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [240]:
max(accumulated_df_conduction.iloc[:,4:].sum(axis=0))

0.184743479295715

In [241]:
# Save comments and ns_accumulated_df_valence to a file, with every row in df is a single line
with open('pdos_sum_valence.dat', 'w') as f:
    for line in comments:
        f.write(line)
    for i in range(len(ns_accumulated_df_valence)):
        f.write(str(ns_accumulated_df_valence.iloc[i,:].values.tolist()).strip('[]') + '\n')


with open('pdos_sum_conduction.dat', 'w') as f:
    for line in comments:
        f.write(line)
    for i in range(len(ns_accumulated_df_conduction)):
        f.write(str(ns_accumulated_df_conduction.iloc[i,:].values.tolist()).strip('[]') + '\n')

## Step 4: Calculate the projectiong into different part of the heterostructure

This part is system specific, for now I just hard-coded it

### 4.0 for APD1_Na

In [242]:
# columns from 37 to 72, 361 to 424 as the Br-per. contribution
# columns from 1 to 36, 329 to 360 as the Br-non. contribution
# columns from 425 to 488 as the O contribution
# columns from 609 to 680 as the Na contribution
# Total is 1 to 680

conduction_perovskite_Pb = ns_accumulated_df_conduction.iloc[:, 37+3:73+3].sum(axis=1) 
conduction_perovskite_Cl = ns_accumulated_df_conduction.iloc[:, 361+3:425+3].sum(axis=1)
conduction_non_perovskite_Pb = ns_accumulated_df_conduction.iloc[:, 1+3:37+3].sum(axis=1)
conduction_non_perovskite_Cl = ns_accumulated_df_conduction.iloc[:, 73+3:361+3].sum(axis=1)
conduction_O = ns_accumulated_df_conduction.iloc[:, 425+3:489+3].sum(axis=1)
conduction_Na = ns_accumulated_df_conduction.iloc[:, 609+3:649+3].sum(axis=1)
conduction_total = ns_accumulated_df_conduction.iloc[:, 1+3:681+3].sum(axis=1)
conduction_other = ns_accumulated_df_conduction.iloc[:, 489+3:609+3].sum(axis=1) + ns_accumulated_df_conduction.iloc[:, 649+3:681+3].sum(axis=1)
#conduction_others = conduction_total - conduction_perovskite - conduction_non_perovskite - conduction_O - conduction_Na - conduction_other

In [243]:
conduction_perovskite_Pb/ conduction_total

0    0.795455
dtype: float64

In [244]:
conduction_perovskite_Cl/ conduction_total

0    0.172146
dtype: float64

In [245]:
conduction_non_perovskite_Cl/ conduction_total

0    0.004846
dtype: float64

In [246]:
conduction_non_perovskite_Pb/ conduction_total

0    0.024476
dtype: float64

In [247]:
# valence bands for APD1_Na
valence_perovskite = ns_accumulated_df_valence.iloc[:, 37+3:73+3].sum(axis=1) + ns_accumulated_df_valence.iloc[:, 361+3:425+3].sum(axis=1)
valence_non_perovskite = ns_accumulated_df_valence.iloc[:, 1+3:37+3].sum(axis=1) + ns_accumulated_df_valence.iloc[:, 73+3:361+3].sum(axis=1)
valence_O = ns_accumulated_df_valence.iloc[:, 425+3:489+3].sum(axis=1)
valence_Na = ns_accumulated_df_valence.iloc[:, 609+3:649+3].sum(axis=1)
valence_total = ns_accumulated_df_valence.iloc[:, 1+3:681+3].sum(axis=1)
valence_other = ns_accumulated_df_valence.iloc[:, 489+3:609+3].sum(axis=1) + ns_accumulated_df_valence.iloc[:, 649+3:681+3].sum(axis=1)
#valence_others = valence_total - valence_perovskite - valence_non_perovskite - valence_O - valence_Na - valence_other

In [248]:
valence_non_perovskite/ valence_total

0    0.123907
dtype: float64

In [249]:
valence_perovskite/ valence_total

0    0.706035
dtype: float64

In [250]:
valence_O/ valence_total

0    0.170042
dtype: float64

### 4.1 for APD2_Pb

In [251]:
# columns from 37 to 72, 105 to 168 as the Br-per. contribution
# columns from 1 to 36, 73 to 104 as the Br-non. contribution
# columns from 169 to 232 as the O contribution
# columns from 609 to 624 as the Li contribution
# Total is 1 to 656

conduction_perovskite = ns_accumulated_df_conduction.iloc[:, 37+3:73+3].sum(axis=1) + ns_accumulated_df_conduction.iloc[:, 105+3:169+3].sum(axis=1)
conduction_non_perovskite = ns_accumulated_df_conduction.iloc[:, 1+3:37+3].sum(axis=1) + ns_accumulated_df_conduction.iloc[:, 73+3:105+3].sum(axis=1)
conduction_O = ns_accumulated_df_conduction.iloc[:, 169+3:233+3].sum(axis=1)
conduction_Li = ns_accumulated_df_conduction.iloc[:, 609+3:625+3].sum(axis=1)
conduction_total = ns_accumulated_df_conduction.iloc[:, 4:660].sum(axis=1)
conduction_other = ns_accumulated_df_conduction.iloc[:, 233+3:609+3].sum(axis=1) + ns_accumulated_df_conduction.iloc[:, 625+3:657+3].sum(axis=1)
conduction_others = conduction_total - conduction_perovskite - conduction_non_perovskite - conduction_O - conduction_Li - conduction_other

In [252]:
conduction_perovskite/ conduction_total

0    0.79546
dtype: float64

In [253]:
valence_perovskite = ns_accumulated_df_valence.iloc[:, 37+3:73+3].sum(axis=1) + ns_accumulated_df_valence.iloc[:, 105+3:169+3].sum(axis=1)
valence_non_perovskite = ns_accumulated_df_valence.iloc[:, 1+3:37+3].sum(axis=1) + ns_accumulated_df_valence.iloc[:, 73+3:105+3].sum(axis=1)
valence_O = ns_accumulated_df_valence.iloc[:, 169+3:233+3].sum(axis=1)
valence_Li = ns_accumulated_df_valence.iloc[:, 609+3:625+3].sum(axis=1)
valence_total = ns_accumulated_df_valence.iloc[:, 5:660].sum(axis=1)
valence_other = ns_accumulated_df_valence.iloc[:, 233+3:609+3].sum(axis=1) + ns_accumulated_df_valence.iloc[:, 625+3:657+3].sum(axis=1)
valence_others = valence_total - valence_perovskite - valence_non_perovskite - valence_O - valence_Li - valence_other

In [254]:
valence_perovskite/valence_total

0    0.241596
dtype: float64

In [255]:
valence_non_perovskite/valence_total

0    0.048398
dtype: float64

In [256]:
valence_Li/valence_total

0    0.000004
dtype: float64

In [257]:
valence_O/valence_total

0    0.000005
dtype: float64

In [258]:
valence_other/valence_total

0    0.709997
dtype: float64

### 4.2 for Hypothetical COCN - Pb-Pb Intergrowth

In [259]:
# columns from 1 to 36, 173 to 236 as the Br-per. contribution
# columns from 37 to 172 as the Br-non. contribution
# columns from 237 to 268 as the O contribution
# columns from 269 to 428 as the other contribution
# Total is 1 to 428


conduction_perovskite = ns_accumulated_df_conduction.iloc[:, 1+3:37+3].sum(axis=1) + ns_accumulated_df_conduction.iloc[:, 173+3:237+3].sum(axis=1)
conduction_non_perovskite = ns_accumulated_df_conduction.iloc[:, 37+3:173+3].sum(axis=1)
conduction_O = ns_accumulated_df_conduction.iloc[:, 237+3:269+3].sum(axis=1)
conduction_other = ns_accumulated_df_conduction.iloc[:, 269+3:429+3].sum(axis=1)
conduction_total = ns_accumulated_df_conduction.iloc[:, 4:432].sum(axis=1)

In [260]:
conduction_non_perovskite/ conduction_total

0    0.798338
dtype: float64

In [261]:
valence_perovskite = ns_accumulated_df_valence.iloc[:, 1+3:37+3].sum(axis=1) + ns_accumulated_df_valence.iloc[:, 173+3:237+3].sum(axis=1)
valence_non_perovskite = ns_accumulated_df_valence.iloc[:, 37+3:173+3].sum(axis=1)
valence_O = ns_accumulated_df_valence.iloc[:, 237+3:269+3].sum(axis=1)
valence_other = ns_accumulated_df_valence.iloc[:, 269+3:429+3].sum(axis=1)
valence_total = ns_accumulated_df_valence.iloc[:, 5:432].sum(axis=1)

In [262]:
valence_perovskite/valence_total

0    0.057775
dtype: float64

### 4.3 for Hypothetical HOCN - Pb-Pb Intergrowth

In [263]:
# columns from 1 to 36, 173 to 236 as the Br-per. contribution
# columns from 37 to 172 as the Br-non. contribution
# columns from 237 to 268 as the O contribution
# columns from 269 to 380 as the Li contribution
# Total is 1 to 380


conduction_perovskite = ns_accumulated_df_conduction.iloc[:, 1+3:37+3].sum(axis=1) + ns_accumulated_df_conduction.iloc[:, 173+3:237+3].sum(axis=1)
conduction_non_perovskite = ns_accumulated_df_conduction.iloc[:, 37+3:173+3].sum(axis=1)
conduction_O = ns_accumulated_df_conduction.iloc[:, 237+3:269+3].sum(axis=1)
conduction_other = ns_accumulated_df_conduction.iloc[:, 269+3:381+3].sum(axis=1)
conduction_total = ns_accumulated_df_conduction.iloc[:, 4:381].sum(axis=1)


valence_perovskite = ns_accumulated_df_valence.iloc[:, 1+3:37+3].sum(axis=1) + ns_accumulated_df_valence.iloc[:, 173+3:237+3].sum(axis=1)
valence_non_perovskite = ns_accumulated_df_valence.iloc[:, 37+3:173+3].sum(axis=1)
valence_O = ns_accumulated_df_valence.iloc[:, 237+3:269+3].sum(axis=1)
valence_other = ns_accumulated_df_valence.iloc[:, 269+3:381+3].sum(axis=1)
valence_total = ns_accumulated_df_valence.iloc[:, 5:381].sum(axis=1)

In [264]:
conduction_non_perovskite/ conduction_total

0    0.92499
dtype: float64

In [265]:
valence_perovskite/valence_total

0    0.121325
dtype: float64